# Calculate Landuse Availabilities

This example shows how `atlite` can deal with landuse restrictions. The script is adjusted from the excellent `atlite` examples that can be found here https://github.com/PyPSA/atlite/tree/master/examples

In [ ]:
import atlite
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

import os
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")


# IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append("./../scripts")

plt.rcParams['figure.figsize'] = [7, 7]

With `geopandas` we retrieve the geographic shapes (in vector format) for Nigeria and Niger.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
countries = ["Nigeria"] # "Niger"]
shapes = world[world.name.isin(countries)].set_index('name')


import math

# For showing cutouts in Africa
# shapes = africa
shapes.plot();
print(shapes.total_bounds)
latmin, lonmin, latmax, lonmax = shapes.total_bounds

#latmin, lonmin, latmax, lonmax = math.floor(latmin), math.ceil(lonmin), math.floor(latmax), math.ceil(lonmax)

We create an `atlite.Cutout` which covers the whole regions and builds the backbone for our analysis. Later, it will enable to retrieve the needed weather data. 

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
%matplotlib inline

In [ ]:
res_t_path = os.path.realpath("resources")+"/profile_solar.nc"
res_t = xr.open_dataset(res_t_path)

In [ ]:
res_t #.values

In [ ]:
res_t = res_t.sel(x=np.arange(latmin,latmax,0.01), y=np.arange(lonmin,lonmax,0.01), method="nearest")

In [ ]:
res_t.potential[:].plot(vmin=1820, vmax=res_t.potential[:].max())
ax.set_xlim(0,20)
ax.set_ylim(0,10)